# HW2 Problem 4 Credit Card Debt

In [1]:
import pandas as pd
import numpy as np

df = pd.DataFrame({"Rate":[0.75, 0.5, 0.01],
                   "MinPayment":[300, 50, 125],
                   "InitialDebt":[30000, 8000, 12000]})

df

,Rate,MinPayment,InitialDebt
0,0.75,300,30000
1,0.50,50,8000
2,0.01,125,12000


In [59]:
import os
import pyomo.environ as pyo
from pyomo.environ import *

from pyomo.opt import SolverFactory

opt = pyo.SolverFactory('glpk')

model = ConcreteModel()

T = [t for t in range(72)]
I = [i for i in range(3)]
D = np.zeros((len(I),len(T)))


model.Y = Var(I, T, within=Binary, initialize=0)
model.X = Var(I, T, within=NonNegativeReals, initialize=0)
model.D = Var(I, T, within=NonNegativeReals, initialize=0)

model.obj = Objective(expr = sum([model.X[i,t] for i in model.X_index_0.data() for t in model.X_index_1.data()]), 
                      sense=minimize)
model.constraints = ConstraintList()

# Debt initialized
for i in model.X_index_0.data():
    model.constraints.add(model.D[i,0] == df.InitialDebt.values[i])


# Payoff all debt
model.constraints.add(sum([model.D[i,71] for i in model.X_index_0.data()]) == 0)

# Initialize all paid debt, initially none
for i in model.X_index_0.data():
    model.constraints.add(model.X[i,0] == 0)
    
# Max monthly payoff
for t in model.X_index_1.data():
    if t >= 2:
        model.constraints.add(sum([model.X[i,t] for i in model.X_index_0.data()]) <= 800)

# Min monthly payoff
for i in model.X_index_0.data():
    for t in model.X_index_1.data():
        if t >= 2:
             model.constraints.add(model.X[i,t] >= df.MinPayment.tolist()[i] + 50*model.Y[i,t])

# Must payoff two debts quick                
for t in model.X_index_1.data():
    if t >= 2:
        model.constraints.add(sum([model.Y[i,t] for i in model.X_index_0.data()]) >= 2)
        
# Rolling debt payoff
for i in model.X_index_0.data():
    for t in model.X_index_1.data():
        if t >= 1:
             model.constraints.add(model.D[i,t] == model.D[i,t-1] * (1 + df.Rate[i]) - model.X[i,t])       


# Solution

In [60]:
# Create a model instance and optimize
instance = model.create_instance()
results = opt.solve(instance, tee=True)
instance.display()

    model; returning a clone of the current model instance.
GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write C:\Users\bmccs\AppData\Local\Temp\tmp_383tdoh.glpk.raw --wglp C:\Users\bmccs\AppData\Local\Temp\tmpbiz3ymjg.glpk.glp
 --cpxlp C:\Users\bmccs\AppData\Local\Temp\tmpm1bzsea_.pyomo.lp
Reading problem data from 'C:\Users\bmccs\AppData\Local\Temp\tmpm1bzsea_.pyomo.lp'...
C:\Users\bmccs\AppData\Local\Temp\tmpm1bzsea_.pyomo.lp:4069: warning: lower bound of variable 'x3' redefined
C:\Users\bmccs\AppData\Local\Temp\tmpm1bzsea_.pyomo.lp:4069: warning: upper bound of variable 'x3' redefined
571 rows, 643 columns, 1489 non-zeros
210 integer variables, all of which are binary
4279 lines were read
Writing problem data to 'C:\Users\bmccs\AppData\Local\Temp\tmpbiz3ymjg.glpk.glp'...
3709 lines were written
GLPK Integer Optimizer, v4.65
571 rows, 643 columns, 1489 non-zeros
210 integer variables, all of which are binary
Preprocessing...
70 hidden packing ineq

In [61]:
P = np.zeros((len(I), len(T)))
D = np.zeros((len(I), len(T)))          
for k,v in instance.X.get_values().items():
    P[k[0], k[1]] = v
for k,v in instance.D.get_values().items():
    D[k[0], k[1]] = v

In [48]:
pd.DataFrame(P)

,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
pd.DataFrame(D)

,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
0,30000.0,400.000000,400.000000,400.00000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,...,463.310311,460.793044,456.387827,448.678697,435.187719,411.578509,370.262391,297.959184,171.428571,0.0
1,8000.0,200.000000,200.000000,200.00000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,...,98.699385,98.049078,97.073617,95.610425,93.415638,90.123457,85.185185,77.777778,66.666667,0.0
2,12000.0,8779.490011,8692.284911,8604.20776,8515.249838,8425.402336,8334.656359,8243.002923,8150.432952,8056.937282,...,1499.053076,1339.043607,1177.434043,1014.208383,849.350467,682.843972,514.672411,344.819135,173.267327,0.0
